<a href="https://colab.research.google.com/github/AlexanderCardarasUCSC/Tabular-Playground-Series---Jun-2021/blob/main/Tabular_Playground_Series_Jun_2021_%5Bacardara%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introuction/Load Data

Code entry for the **Tabular Playground Series - Jun 2021** kaggle competition.

https://www.kaggle.com/c/tabular-playground-series-jun-2021/overview



Inspiration for the code from the keras functional_api guide.

https://keras.io/guides/functional_api/.


In [ ]:
# mount google drive to download dataset
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# create a folder to store dataset files
!mkdir /content/dataset

# copy dataset from google drive
!cp /content/gdrive/MyDrive/kaggle/train.csv /content/dataset
!cp /content/gdrive/MyDrive/kaggle/test.csv /content/dataset

In [ ]:
import pandas as pd
import numpy as np

raw_dataset = pd.read_csv("/content/dataset/train.csv", header=0)
dataset_train = raw_dataset.to_numpy()[0:140_000]
dataset_validation = raw_dataset.to_numpy()[140_000:]

print(dataset_train.shape)
print(dataset_validation.shape)

raw_testset = pd.read_csv("/content/dataset/test.csv", header=0)
raw_testset = raw_testset.to_numpy()
print(raw_testset.shape)

### Training Loop

In [19]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

NUMBER_OF_CLASSES = 9
SUMMARIZE_AFTER = 1

# applying to_categorical is currently causing an error, 
# TODO: implement it correctly
def string_to_categorical(labels):
  new_labels = []
  for label in labels:
    # we want the 6th character of each class label string ie 'Class_7' should 
    # convert to '7'

    # to_categorical expects values to start from 0, dataset is in range [1,9]
    # we must subtract 1 from class labels for training to get the range [0,8]
    new_labels.append(int(label[0][6])-1)
  new_labels = np.asarray(new_labels)
  # return to_categorical(new_labels, NUMBER_OF_CLASSES)
  return new_labels

def generate_training_samples(dataset, n_samples):
  rows = [np.random.randint(0,dataset.shape[0]) for _ in range(n_samples)]
  samples = dataset[rows]

  train_x = samples[:,1:-1].astype("float32")
  train_y = samples[:,-1:]
  train_y = string_to_categorical(train_y).astype("float32")
  return train_x, train_y

# print accuracy of holdout dataset
def summarize_performance(dataset_validation, model, epoch):
  train_x, train_y = generate_training_samples(dataset_validation, dataset_validation.shape[0])
  _loss, _accuracy = model.evaluate(train_x, train_y)
  validation_loss.append(_loss)

def train(dataset_train, dataset_validation, model, n_epochs, batch_size):
  iterations_per_epoch = dataset_train.shape[0]
  batches_per_epoch = iterations_per_epoch//batch_size
  for epoch in range(n_epochs):
    temp_train_loss = []
    for batch in range(batches_per_epoch):
        train_x, train_y = generate_training_samples(dataset_train, batch_size)

        _loss, _accuracy = model.train_on_batch(train_x, train_y)
        temp_train_loss.append(_loss)

        # print metrics every once in a while
        if batch % 200 == 0:
          print(">%d %d/%d, %.3f, %.3f"%(epoch, batch, batches_per_epoch, _loss, _accuracy))
    if (epoch+1) % SUMMARIZE_AFTER == 0:
      train_loss.append(sum(temp_train_loss)/len(temp_train_loss))
      summarize_performance(dataset_validation, model, epoch)



### Models

In [20]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

def define_model():
  input_layer = Input(shape=(75,), name="First_Layer")

  output_layer = Dense(128, activation="relu")(input_layer)
  output_layer = Dropout(0.4)(output_layer)
  output_layer = Dense(64, activation="relu")(output_layer)
  output_layer = Dense(NUMBER_OF_CLASSES, activation="softmax")(output_layer)
  
  model = Model(inputs=input_layer, outputs=output_layer, name="Simple_Classification_Model")

  loss = SparseCategoricalCrossentropy()
  opt = Adam(learning_rate=0.0005, beta_1=0.5)
  model.compile(loss=loss, optimizer=opt, metrics=["accuracy"])
  return model

### Train Models

In [ ]:
train_loss, validation_loss = [], []

n_epochs = 7
batch_size = 100

model = define_model()
model.summary()
train(dataset_train, dataset_validation, model, n_epochs, batch_size)

### Visualize losses

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x = [SUMMARIZE_AFTER*i for i in range(len(train_loss))]
plt.plot(x, train_loss)
plt.plot(x, validation_loss)
plt.show()

### Run Test

In [121]:
import pandas as pd


def predict_class(model, features):
  features = np.expand_dims(features, axis=0)
  return model.predict(features)

def predict_testset(model, testset):
  labels = ["id","Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"]
  
  results = []
  ids = testset[:,0]
  ids = ids.reshape((len(ids),1));
  ids = pd.DataFrame(data=ids, columns=[labels[0]])

  data = testset[:,1:]
  results = model.predict(data)
  results = pd.DataFrame(data=results, columns=labels[1:])

  final = pd.concat([ids, results], axis=1)
  # final = np.hstack((ids,results))

  # return pd.DataFrame(data=final, columns=labels)
  return final


results = predict_testset(model, raw_testset)
results.to_csv("submission.csv", index=False)
# print(results)

### UMAP

In [ ]:
!pip install umap-learn
!pip install babyplots
!pip install hdbscan 

In [ ]:
import umap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from babyplots import Babyplot
import hdbscan



dataset = raw_dataset.to_numpy()[:,1:-1]
labels = raw_dataset.to_numpy()[:,-1]

dataset = StandardScaler().fit_transform(dataset)

le = preprocessing.LabelEncoder()
class_names = ["Class_1", "Class_2", "Class_3", "Class_4", "Class_5", "Class_6", "Class_7", "Class_8", "Class_9"]
le.fit(class_names)
labels = le.transform(labels)

print(labels)
print(dataset)
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.30)
reducer2d = umap.UMAP(n_neighbors=20,
                        min_dist=0.0,
                        n_components=2,
                        random_state=42, metric='euclidean', target_weight=0)

reducer3d = umap.UMAP(n_neighbors=20,
                      min_dist=0.0,
                      n_components=3,
                      random_state=42, metric='euclidean', target_weight=0)

# mapper2d = reducer2d.fit(X_train, y_train)
mapper3d = reducer3d.fit(X_train, y_train)

# embedding2d_train = mapper2d.transform(X_train)
embedding3d_train = mapper3d.transform(X_train)



# embedding2d_test = mapper2d.transform(X_test)
embedding3d_test = mapper3d.transform(X_test)

# embedding2d = embedding2d_train.tolist() + embedding2d_test.tolist()
embedding3d = embedding3d_train.tolist() + embedding3d_test.tolist()
# y_test 
print(y_test.shape)
# y_test = np.where(y_test >= 1, 'RI', 'DE')
y_plot = y_train.tolist() + y_test.tolist()
bp = Babyplot(background_color="#ffffddff", )
bp.add_plot(embedding3d, 'pointCloud', 'categories', y_plot, {
    'colorScale': 'Viridis',
    'showLegend': True,
    'folded': True,
    'size': 5,
    'showAxes': [True, True, True],
    'axisLabels': ['X', 'Y', 'Z'],
    'showTickLines': [[True, True], [True, True], [True, True]]
})
bp.save_as_html(r'/content/test3.html')


In [9]:
import hdbscan
from sklearn.model_selection import train_test_split


dataset = raw_dataset.to_numpy()[:,1:-1]
labels = raw_dataset.to_numpy()[:,-1]

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.30)

In [ ]:
hdbscan_labels = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=500).fit_predict(X_train, y_train)